## Set up the environment

In [5]:
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()
model_package_arn='opinion-mining-sentiment-analy-c080063b1353c8be82f06cc648a187a7'
s3_base_path='s3://[your s3 path]'

## Realtime Inference

In [6]:
from sagemaker import ModelPackage
model_name='opinion-mining-sentiment-analysis'
model = ModelPackage(name=model_name,
                     role=role,
                     model_package_arn=model_package_arn,
                     sagemaker_session=sagemaker_session)

In [7]:
endpoint_name='opinion-mining-sentiment-analysis-edp'

In [8]:
predictor = model.deploy(1, 'ml.p2.xlarge', endpoint_name=endpoint_name)

In [9]:
# connect to the endpoint
predictor = sage.predictor.RealTimePredictor(
    endpoint_name,
    sagemaker_session=sagemaker_session
)

In [10]:
test_json = '{"content": "The park was very fun.  It was an excellent vacation."}'
res = predictor.predict(test_json)
    
print(res)

In [11]:
# Delete endpoint
predictor.delete_endpoint()

## Batch Transform Job

In [12]:
s3_output_path=s3_base_path + '/output/'
transformer = sage.transformer.Transformer(
    base_transform_job_name='opinion-mining-sentiment-analysis-batch-transform',
    model_name=model_name,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=s3_output_path,
    sagemaker_session=sagemaker_session
)

In [14]:
transformer.transform(
    s3_base_path + '/input/input.json'
)

# wait until prediction job is completed
transformer.wait()


#### Inspect the Batch Transform Output in S3

In [15]:
s3_client = sage.s3.S3Downloader
s3_client.read_file(s3_base_path + '/output/input.json.out')